In [2]:
import numpy as np



In [8]:
def psi(x):
    assert x >= 0
    return 1/(1+x)

def phi(x):
    assert x >= 0 & x < 1
    return 1/(1-x)

def forward_swap(d_in, q, g):
    assert q[0] > 0
    d_out = (1 - g) * ( q[1]/q[0] ) * psi(d_in/q[0]) * d_in
    return d_out

def backward_swap(d_out, q, f):
    assert q[1] > 0
    d_in = (1/(1-f)) * ( q[0]/q[1] ) * phi(d_out/q[1]) * d_out
    return d_in


In [9]:
F = 0.003
G = 0.003

Q_ab = (100, 100)
Q_ac = (200, 200)
Q_bc = (250, 250)

In [10]:
# a->b route
D_a = 10
D_b1 = forward_swap(d_in=D_a, q=Q_ab, g=G)

# a->c->b route
D_c = forward_swap(d_in=D_a, q=Q_ac, g=G)
D_b2 = forward_swap(d_in=D_c, q=(Q_bc[1],Q_bc[0]), g=G)

print(D_b1, D_c, D_b2)

9.063636363636364 9.495238095238093 9.120352699379746


In [11]:
# cycle c -> b -> a -> c

C_b = forward_swap(d_in=D_c, q=Q_bc, g=G)
